In [3]:
import pandas  as pd 
import numpy as np
import torch
import torch.nn  as nn
import torch.nn.functional as F
from torch.autograd import Variable
import psycopg2

In [39]:
start = "2014-01-01"
end = "2020-08-20"
conn = psycopg2.connect(host="slave",
                        database="testdb1",
                        user="postgres",
                        password="reallyStrongPwd123")
cursor = conn.cursor()
symbols = ("MSFT", "GOOGL", "FB")
query2 = f""" SELECT * from incertae."StockData" where symbol in {symbols} and date between '{start}' AND '{end}' """
df = pd.read_sql_query(query2, con=conn)
df.set_index(["date", "symbol"], inplace=True)
df = df.unstack()
df.columns = df.columns.values
y= df[('high', 'FB')].shift(1)
x=df.drop(('high', 'FB'),axis=1)
y = y.fillna(0)

N, D_in, D_out = 100, df.shape[1]-1, 1
class LinearRegression(nn.Module):
    def __init__(self, input_size, output_size):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(input_size,output_size)
    
    def forward(self, X):
        out = self.linear(X)
        return out
model = LinearRegression(D_in, D_out)
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)
for epoch in range(100):
    inputs = Variable(torch.from_numpy( x.values)).float()
    labels = Variable(torch.from_numpy( y.values)).float()
    optimizer.zero_grad()

    outputs = model(inputs)

    loss = criterion(outputs,labels)
    loss.backward()
    optimizer.step()


In [25]:
inputs.shape

torch.Size([1671, 17])

In [44]:
    predicted = model(Variable(torch.from_numpy(x.values)).float())


In [43]:
with torch.no_grad():
    predicted = model(Variable(torch.from_numpy(x.values)).float())
    print(predicted)

tensor([[nan],
        [nan],
        [nan],
        ...,
        [nan],
        [nan],
        [nan]])


In [38]:
loss

tensor(1.1586e+14, grad_fn=<MseLossBackward>)

In [47]:
# create dummy data for training
x_values = [i for i in range(11)]
x_train = np.array(x_values, dtype=np.float32)
x_train = x_train.reshape(-1, 1)

y_values = [2*i + 1 for i in x_values]
y_train = np.array(y_values, dtype=np.float32)
y_train = y_train.reshape(-1, 1)
class linearRegression(torch.nn.Module):
    def __init__(self, inputSize, outputSize):
        super(linearRegression, self).__init__()
        self.linear = torch.nn.Linear(inputSize, outputSize)

    def forward(self, x):
        out = self.linear(x)
        return out
inputDim = 1        # takes variable 'x' 
outputDim = 1       # takes variable 'y'
learningRate = 0.01 
epochs = 100

model = linearRegression(inputDim, outputDim)
##### For GPU #######
if torch.cuda.is_available():
    model.cuda()
criterion = torch.nn.MSELoss() 
optimizer = torch.optim.SGD(model.parameters(), lr=learningRate)
for epoch in range(epochs):
    # Converting inputs and labels to Variable
    if torch.cuda.is_available():
        inputs = Variable(torch.from_numpy(x_train).cuda())
        labels = Variable(torch.from_numpy(y_train).cuda())
    else:
        inputs = Variable(torch.from_numpy(x_train))
        labels = Variable(torch.from_numpy(y_train))

    # Clear gradient buffers because we don't want any gradient from previous epoch to carry forward, dont want to cummulate gradients
    optimizer.zero_grad()

    # get output from the model, given the inputs
    outputs = model(inputs)

    # get loss for the predicted output
    loss = criterion(outputs, labels)
    print(loss)
    # get gradients w.r.t to parameters
    loss.backward()

    # update parameters
    optimizer.step()

    print('epoch {}, loss {}'.format(epoch, loss.item()))

tensor(128.7580, grad_fn=<MseLossBackward>)
epoch 0, loss 128.7580108642578
tensor(10.5120, grad_fn=<MseLossBackward>)
epoch 1, loss 10.5120267868042
tensor(0.8670, grad_fn=<MseLossBackward>)
epoch 2, loss 0.8669695258140564
tensor(0.0801, grad_fn=<MseLossBackward>)
epoch 3, loss 0.08014492690563202
tensor(0.0159, grad_fn=<MseLossBackward>)
epoch 4, loss 0.015860892832279205
tensor(0.0105, grad_fn=<MseLossBackward>)
epoch 5, loss 0.010513232089579105
tensor(0.0100, grad_fn=<MseLossBackward>)
epoch 6, loss 0.00997408851981163
tensor(0.0098, grad_fn=<MseLossBackward>)
epoch 7, loss 0.009828353300690651
tensor(0.0097, grad_fn=<MseLossBackward>)
epoch 8, loss 0.009715796448290348
tensor(0.0096, grad_fn=<MseLossBackward>)
epoch 9, loss 0.009606998413801193
tensor(0.0095, grad_fn=<MseLossBackward>)
epoch 10, loss 0.009499729610979557
tensor(0.0094, grad_fn=<MseLossBackward>)
epoch 11, loss 0.009393606334924698
tensor(0.0093, grad_fn=<MseLossBackward>)
epoch 12, loss 0.00928875058889389
tenso